<a href="https://colab.research.google.com/github/ma850419/Various_scripts/blob/main/ayrclass3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='velvety-ring-328419')

In [ ]:
# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)

In [ ]:
#imageROI =ee.FeatureCollection('users/mohamadawadlebanon/borderutm36n')
imageROI =ee.FeatureCollection('users/mohamadawadlebanon/Iranborder')
#trainingPolys = ee.FeatureCollection('users/mohamadawadlebanon/fieldsamples_2019_36N')
#evalPolys=ee.FeatureCollection('users/mohamadawadlebanon/fieldsamples_2019_36N')
training = ee.FeatureCollection('users/mohamadawadlebanon/crop2')

In [ ]:
record_count = training.size().getInfo()
print(f"Number of records: {record_count}")

# Convert FeatureCollection to a list and print all features
features = training.toList(record_count)
for i in range(record_count):
    feature = ee.Feature(features.get(i))
    print(feature.getInfo())  # Prints full feature properties

In [ ]:
import folium

# Initialize Earth Engine

# Use Sentinel-2 surface reflectance data.
s2sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Cloud masking function for Sentinel-2.
def maskS2sr(image):
    qa = image.select('QA60')  # Sentinel-2 cloud mask band
    mask = qa.eq(0)  # Keep only pixels where QA60 is clear
    return image.updateMask(mask).divide(10000)  # Scale reflectance values

# Clip function
def clipimage(image):
    return image.clip(imageROI)

# Define Image ROI (Replace with a valid region)
#imageROI = ee.Geometry.Rectangle([35, 30, 40, 35])  # Example ROI

# Create a cloud-masked median composite for July 2019
image = s2sr.filterDate('2024-05-01', '2024-05-31').map(clipimage).map(maskS2sr).median()
print(image.getInfo())
# Use folium to visualize the imagery.
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})  # RGB composite
map = folium.Map(location=[28, 53], zoom_start=6)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Sentinel-2 Composite',
).add_to(map)

'''mapid = image.getMapId({'bands': ['B11'], 'min': 0, 'max': 0.3})  # Thermal (SWIR Band)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='SWIR Band (B11)',
).add_to(map)

map.add_child(folium.LayerControl())'''

# Display map
map


In [ ]:
image_np = geemap.ee_to_numpy(image, imageROI, bands=['B4', 'B3', 'B2'], scale=55)  # 10m resolution


In [ ]:
import geemap
import numpy as np

# Extract image bands
region = imageROI  # Define a bounding box (Iran region)

# Convert to NumPy with specified region
image_np = geemap.ee_to_numpy(image,imageROI, bands=['B4', 'B3', 'B2'], scale =60)

'''training_image = training.reduceToImage(
    properties=['class'],
    reducer=ee.Reducer.first()
)'''
samples_image = training.reduceToImage(['class'], ee.Reducer.mode())  # Convert points to raster
# Change 'class' to 'mode' to match the band name created by reduceToImage
samples_np = geemap.ee_to_numpy(samples_image, region=region, bands=['mode'], scale =60)  # Extract pixels

# Define region (Ensure it's bounded)

# Convert to NumPy
#samples_np = geemap.ee_to_numpy(training_image, region=region, scale=40)

In [ ]:
print(np.unique(samples_np))

In [ ]:
#image_np = np.expand_dims(image_np, axis=-1)  # Converts to (height, width, 1)
image_np = np.expand_dims(image_np, axis=0)  # Becomes (1, 1439, 1825, 3)
samples_np = np.expand_dims(samples_np, axis=0)  # Becomes (1, 1439, 1825, 1)

print(image_np.shape)
#print(image_np)  # Inspect sample values

print(samples_np.shape )

In [ ]:
new_height = image_np.shape[1] - (image_np.shape[1] % 2)  # Ensure divisible by 2
new_width = image_np.shape[2] - (image_np.shape[2] % 2)

image_np = image_np[:, :new_height, :new_width, :]  # Trim image size
samples_np = samples_np[:, :new_height, :new_width, :]  # Match target size
print(image_np.shape)
#print(image_np)  # Inspect sample values

print(samples_np.shape )

In [ ]:
print("Unique values in samples_np before encoding:", np.unique(samples_np))


In [ ]:
# image_np = np.expand_dims(image_np, axis=-1)  # Converts to (height, width, 1)
#image_np = np.expand_dims(image_np, axis=0)  # Becomes (1, 1439, 1825, 3)
#samples_np = np.expand_dims(samples_np, axis=0)  # Becomes (1, 1439, 1825, 1)

#print(image_np.shape)
#print(image_np)  # Inspect sample values

#print(samples_np.shape )

# One-hot encode the target labels
# Subtract 1 from samples_np to make class indices 0-based

# One-hot encode the target labels
samples_one_hot = tf.keras.utils.to_categorical(samples_np, num_classes=21)

# Remove the extra dimension added by to_categorical if necessary
if samples_one_hot.ndim == 5 and samples_one_hot.shape[-2] == 1:
    samples_one_hot = np.squeeze(samples_one_hot, axis=-2)

# Convert to integer format
samples_one_hot = samples_one_hot.astype(int)

print("Updated shape:", samples_one_hot)
print("Unique values:", np.unique(samples_one_hot))  # Should contain only 0s and 1s

In [ ]:
# Ensure samples_np has correct dimensions
print("samples_np shape:", samples_np.shape)  # Expected: (1, 1438, 1824)

# Check samples_one_hot dimensions
print("samples_one_hot shape:", samples_one_hot.shape)  # Expected: (1, 1438, 1824, 21)

# Create mask that matches the exact shape of samples_one_hot
mask = np.repeat(samples_np == 0, repeats=samples_one_hot.shape[-1], axis=-1)  # Shape: (1, 1438, 1824, 21)

# Apply mask to remove No Data pixels
samples_one_hot[mask] = 0

# Verify changes
print("Updated shape:", samples_one_hot.shape)
print("Unique values:", np.unique(samples_one_hot))  # Should contain only 0s and 1s


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Sum pixel occurrences for each class (excluding spatial dimensions)
class_distribution = np.sum(samples_one_hot, axis=(0, 1))  # Shape: (21,)


# Remove No Data class (class 0)
class_distribution_filtered = class_distribution[1:]  # Excludes class 0
print("Filtered class distribution:", class_distribution_filtered)


# Plot histogram
plt.figure(figsize=(10, 6))
plt.bar(range(1, 21), class_distribution_filtered, color='blue', alpha=0.7)
plt.xlabel("Class Labels (excluding No Data)")
plt.ylabel("Frequency of '1' Values")
plt.title("Class Distribution in samples_one_hot (No Data Removed)")
plt.xticks(range(1, 21))
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.show()

# Print detailed class counts
for cls, count in enumerate(class_distribution_filtered, start=1):  # Start from class 1
    print(f"Class {cls}: {count} pixels")


In [ ]:
image_np = np.expand_dims(image_np, axis=0)  # Becomes (1, 1439, 1825, 3)
samples_one_hot = np.expand_dims(samples_one_hot, axis=0)  # Becomes (1, 1439, 1825, 1)

print(image_np.shape)
#print(image_np)  # Inspect sample values

print(samples_one_hot.shape )

In [ ]:
new_height = image_np.shape[1] - (image_np.shape[1] % 2)  # Ensure divisible by 2
new_width = image_np.shape[2] - (image_np.shape[2] % 2)

image_np = image_np[:, :new_height, :new_width, :]  # Trim image size
samples_one_hot = samples_one_hot[:, :new_height, :new_width, :]  # Match target size
print(image_np.shape)
#print(image_np)  # Inspect sample values

print(samples_one_hot.shape )

In [ ]:
print(samples_one_hot)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow import keras

# Define U-Net Model for 20-class classification
model = tf.keras.Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(image_np.shape[1], image_np.shape[2], image_np.shape[3])),
    MaxPooling2D((2, 2)),
    UpSampling2D((2, 2)),
    Conv2D(20, (1, 1), activation='softmax', padding='same')  # Adjusted for multi-class classification
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model (using GEE sample data)
model.fit(image_np, samples_one_hot[:,:,:,1:21], epochs=10, batch_size=32)


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dropout, BatchNormalization

model = keras.Sequential([
    # Encoder
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(image_np.shape[1], image_np.shape[2], image_np.shape[3])),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Decoder
    UpSampling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),

    UpSampling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),

    UpSampling2D((2, 2)),
    Conv2D(20, (1, 1), activation='softmax', padding='same')  # Multi-class classification
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(image_np, samples_one_hot[:,:,:,1:21], epochs=10, batch_size=16)#, class_weight=class_weights)


In [ ]:
print(image_np.shape)
#print(image_np)  # Inspect sample values

print(samples_one_hot.shape )
print("Image shape:", image_np.shape)
print("Labels shape:", samples_one_hot[:,:,:,1:21].shape)

In [ ]:
predictions = model.predict(image_np)
print(predictions)

In [ ]:
import numpy as np

# Get the band index with the highest value per pixel
class_predictions = np.argmax(predictions, axis=-1)  # Shape: (1438, 1824)

# Adjust indices to match bands 1-20 (instead of 0-19)
class_predictions += 1

# Verify results
print("Updated shape:", class_predictions.shape)  # Should be (1438, 1824)
print("Unique classes:", np.unique(class_predictions))  # Should contain values [1, 2, ..., 20]


In [ ]:
viz_params = {
    'min': 1,
    'max': 20,
    'palette': [
        'red', 'yellow', 'green', 'blue', 'purple', 'orange', 'brown', 'cyan', 'pink',
        'magenta', 'lime', 'gray', 'black', 'white', 'gold', 'navy', 'teal', 'maroon',
        'turquoise', 'violet'
    ]
}


In [ ]:
predicted_image = predicted_image.setDefaultProjection('EPSG:4326', None, 30)  # WGS84 with 30m scale



In [ ]:
print(predictions_scaled.shape)
  # Ensure it prints valid geometric coordinates
pixels = ee.Array(predictions_scaled.tolist())  # Convert NumPy array to EE array
predicted_image = ee.Image(pixels).clip(imageROI)  # Clip to the target region


In [ ]:
print(predicted_image.projection().getInfo())  # Confirms raster properties


In [ ]:
print(predicted_image)



In [ ]:
Map.addLayer(predicted_image, {'min': 1, 'max': 20, 'palette': ['red', 'blue', 'green']}, "Predicted Image")


In [ ]:
Map = geemap.Map()
Map.addLayer(predictions_scaled, viz_params, "U-Net Classified Image")
Map.centerObject(region, 10)  # Adjust zoom level
Map


In [ ]:
predicted_image_ee = geemap.numpy_to_ee(predictions_scaled, imageROI)

# Set the projection and scale explicitly
predicted_image_ee = predicted_image_ee.setDefaultProjection(
    crs='EPSG:4326',  # Specify a common projection
    scale=60          # Use the same scale as when converting to numpy
)


Map = geemap.Map()
# Add the Earth Engine image to the map
Map.addLayer(predicted_image_ee, {}, "U-Net Classified Image")
Map.centerObject(imageROI.geometry(), 10)  # Adjust zoom level and use the region's geometry
Map